In [1]:
import numpy as np
import pandas as pd
import gc
import time
import random
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from keras import backend as K
from keras.models import Sequential, load_model
from keras.optimizers import Adam, Nadam
from keras.initializers import glorot_uniform, lecun_uniform
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, Dropout, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit



Using TensorFlow backend.


In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [3]:
df_trans = pd.read_csv(f'{folder_path}train_transaction.csv').iloc[:6000,:]
df_id = pd.read_csv(f'{folder_path}train_identity.csv').iloc[:6000,:]
df_train = pd.merge(df_trans, df_id, how='left', on='TransactionID')
del df_trans
del df_id

df_trans = pd.read_csv(f'{folder_path}test_transaction.csv').iloc[:1000,:]
df_id = pd.read_csv(f'{folder_path}test_identity.csv').iloc[:1000,:]
df_test = pd.merge(df_trans, df_id, how='left', on='TransactionID')
del df_trans
del df_id

na_exist_columns = []
for col in df_test.columns:
    if df_train[col].isnull().sum()>0:
        na_exist_columns.append(col)
    else:
        if df_test[col].isnull().sum()>0:
            na_exist_columns.append(col)
            
# train_isnull = df_train[na_exist_columns].isnull().astype(int)
# train_isnull.columns = [f'{col}_isna' for col in train_isnull.columns]
# test_isnull = df_test[na_exist_columns].isnull().astype(int)
# test_isnull.columns = [f'{col}_isna' for col in test_isnull.columns]

for col in df_test.columns:
    try:
        if not is_numeric_dtype(df_test[col]):
            le = LabelEncoder()
            le.fit(df_train[col].fillna('').tolist() + df_test[col].fillna('').tolist())
            df_train[col] = le.transform(df_train[col].fillna(''))
            df_test[col] = le.transform(df_test[col].fillna(''))
        else:
            min_ = np.min([df_train[col].dropna().min(), df_test[col].dropna().min()])
            max_ = np.max([df_train[col].dropna().max(), df_test[col].dropna().max()])
            replace_value = min_- (max_ - min_)*.1
            df_train[col] = df_train[col].fillna(replace_value)
            df_test[col] = df_test[col].fillna(replace_value)
    except:
        raise Exception(f'{col}-{df_train[col].dtype}')
        
for col in df_train.columns:
     if not is_numeric_dtype(df_train[col]):
        print(col, df_train[col].dtype)
        
for col in df_test.columns:
     if not is_numeric_dtype(df_test[col]):
        print(col, df_test[col].dtype)
        
        
scaler = MinMaxScaler()
scaler.fit(df_train[df_test.columns.tolist()].values)

X_train = pd.DataFrame(scaler.transform(df_train[df_test.columns.tolist()].values)*255).astype(int).values
# X_train = pd.concat([X_train, train_isnull], axis=1).values

y_train = df_train['isFraud'].values

X_test = pd.DataFrame(scaler.transform(df_test[df_test.columns.tolist()].values)*255).astype(int).values
# X_test = pd.concat([X_test, test_isnull], axis=1).values

del df_train
del df_test

In [3]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip').iloc[:6000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip').iloc[:1000,:]

columns = df_train.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')
columns.remove('isFraud')

X_train = df_train[columns].values
y_train = df_train['isFraud'].values
X_test = df_test[columns].values


In [4]:
X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)

In [5]:
number_of_folds = 5
# folds = TimeSeriesSplit(n_splits=number_of_folds)
folds = KFold(n_splits=number_of_folds, shuffle=False, random_state=42)

In [6]:
def EarlyStop(patience):
    return EarlyStopping(monitor = "val_loss",
                          min_delta = 0,
                          mode = "min",
                          verbose = 1, 
                          patience = patience)

def ModelCheckpointFull(model_name):
    return ModelCheckpoint(model_name, 
                            monitor = 'val_loss', 
                            verbose = 1, 
                            save_best_only = True, 
                            save_weights_only = False, 
                            mode = 'min', 
                            period = 1)

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

filters = 96

# Define CNN 1D model
def create_model():
    model = Sequential()
    model.add(Conv1D(filters, 2, activation = 'relu', input_shape=(input_shape, 1), kernel_initializer = glorot_uniform(seed = seed)))
    model.add(BatchNormalization())       
    model.add(Dropout(0.25))
    
    for _ in range(2):
        model.add(Conv1D(filters, 1, activation = 'relu', kernel_initializer = glorot_uniform(seed = seed)))
        model.add(BatchNormalization())       
        model.add(Dropout(0.25))    
    
    model.add(Flatten())
    model.add(Dense(1024, activation = 'relu', kernel_initializer = glorot_uniform(seed = seed)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(128, activation = 'relu', kernel_initializer = glorot_uniform(seed = seed)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = glorot_uniform(seed = seed)))
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr = 0.0005), metrics = [auc])
    
    return model

In [8]:
# Random Seed
seed = 12345
np.random.seed(seed)
random.seed(seed)

# Constants
epochs = 200
batch_size = 1024

# Input Shape
input_shape = X_train.shape[1]

# Arrays to store predictions
oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])

# X_test = np.expand_dims(X_test, 2)

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X_train, y_train)):
    train_x, train_y = X_train[train_idx], y_train[train_idx]
    valid_x, valid_y = X_train[valid_idx], y_train[valid_idx]
    
    # Reshape
#     train_x = np.expand_dims(train_x, 2)
#     valid_x = np.expand_dims(valid_x, 2)

    print('Running Fold: ' + str(n_fold))

    # CNN 1D model
    model = create_model()
    model.fit(train_x, train_y, 
                validation_data=(valid_x, valid_y), 
                epochs=epochs, 
                batch_size=batch_size, 
                verbose=0,
                callbacks=[EarlyStop(20)])#, ModelCheckpointFull('model.h5')

    # Delete Model
#     del model
#     gc.collect()

    # Reload Best Saved Model
#     model = load_model('model.h5')

    # OOF Predictions
    oof_preds[valid_idx] = model.predict(valid_x).reshape(-1,)
    
    # Submission Predictions
    predictions = model.predict(X_test).reshape(-1,)
    sub_preds += predictions / number_of_folds

    # Fold AUC Score
    print('Fold %2d AUC : %.6f' % (n_fold, roc_auc_score(valid_y, oof_preds[valid_idx])))        

    # Cleanup 
    del model, train_x, train_y, valid_y, valid_x
    K.clear_session()
    gc.collect

print('Full AUC score %.6f' % roc_auc_score(y_train, oof_preds))

Running Fold: 0
Epoch 00028: early stopping
Fold  0 AUC : 0.852643
Running Fold: 1
Epoch 00032: early stopping
Fold  1 AUC : 0.744725
Running Fold: 2
Epoch 00026: early stopping
Fold  2 AUC : 0.750634
Running Fold: 3
Epoch 00025: early stopping
Fold  3 AUC : 0.776603
Running Fold: 4
Epoch 00027: early stopping
Fold  4 AUC : 0.879061
Full AUC score 0.735906


In [24]:
oof_preds.shape

(6000,)

In [25]:
predictions.shape

(1000,)